In [1]:
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.callbacks import TensorBoard

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [6]:
model = keras.models.load_model('/content/gdrive/My Drive/model3Pa.h5')
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 124, 38, 128)      1280      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 62, 19, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 60, 17, 64)        73792     
_________________________________________________________________
flatten_21 (Flatten)         (None, 65280)             0         
_________________________________________________________________
dense_32 (Dense)             (None, 5)                 326405    
Total params: 401,477
Trainable params: 401,477
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.layers[1].output

<tf.Tensor 'max_pooling2d_24/MaxPool_3:0' shape=(None, 62, 19, 128) dtype=float32>

In [ ]:
import os
import numpy as np

sample_path = '/content/gdrive/My Drive/Speech_Negatives/Clips/'

Gina1 = "Best of Holt and Gina Brooklyn Nine-Nine.mp31.mp3"
Gina2 = "Best of Holt and Gina Brooklyn Nine-Nine.mp32.mp3"
Cody1 = "CAN HAWK OF THE FOG REPLACE HERRSCHER THUNDER MEI!! HONKAI IMPACT 3.mp3318.mp3"
Cody2 = "CAN HAWK OF THE FOG REPLACE HERRSCHER THUNDER MEI!! HONKAI IMPACT 3.mp3319.mp3"

INPUT_SHAPE = (126,40)
samples = [Gina1, Gina2, Cody1, Cody2]
mfccs = np.zeros((len(samples), INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float32)

count = 0
for sample in samples:
    y, sr = librosa.load(sample_path + sample)
    mfcc = librosa.feature.mfcc(y=y, sr=16000, hop_length=128, n_fft=256, n_mfcc=20)[:, 23:149]
    mfcc_delta = librosa.feature.delta(mfcc)[:10, :]
    mfcc_double_delta = librosa.feature.delta(mfcc, order=2)[:10, :]
    mfccs[count, :, :20] = mfcc.T
    mfccs[count, :, 20:30] = mfcc_delta.T
    mfccs[count, :, 30:] = mfcc_double_delta.T
    count += 1

mfccs = np.expand_dims(mfccs, 3)
print(mfccs.shape)

In [ ]:
model.predict(mfccs)

array([[0.00000000e+00, 1.00000000e+00, 7.64379308e-22, 1.07942715e-32,
        0.00000000e+00],
       [0.00000000e+00, 1.16754766e-11, 1.00000000e+00, 5.93226051e-26,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 3.58894967e-21,
        1.60851835e-26],
       [0.00000000e+00, 1.73574436e-32, 1.00000000e+00, 3.14418222e-22,
        4.23876010e-08]], dtype=float32)

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
from keras import backend as K

def get_layer_outputs(sample, model):
    test_image = sample
    outputs    = [layer.output for layer in model.layers]          # all layer outputs
    comp_graph = [K.function([model.input]+ [K.learning_phase()], [output]) for output in outputs]  # evaluation functions

    # Testing
    layer_outputs_list = [op([test_image, 1.]) for op in comp_graph]
    layer_outputs = []

    for layer_output in layer_outputs_list:
        print(layer_output[0][0].shape, end='\n-------------------\n')
        layer_outputs.append(layer_output[0][0])

    return layer_outputs

def plot_layer_outputs(layer_number, sample, model):    
    layer_outputs = get_layer_outputs(sample, model)

    x_max = layer_outputs[layer_number].shape[0]
    y_max = layer_outputs[layer_number].shape[1]
    n     = layer_outputs[layer_number].shape[2]

    L = []
    for i in range(n):
        L.append(np.zeros((x_max, y_max)))

    for i in range(n):
        for x in range(x_max):
            for y in range(y_max):
                L[i][x][y] = layer_outputs[layer_number][x][y][i]


    for img in L:
        plt.figure()
        plt.imshow(img, interpolation='nearest')

In [9]:
temp_model = keras.Model(inputs=model.input, outputs=model.get_layer('flatten_21').output)
temp_model.summary()
temp_model.save("/content/gdrive/My Drive/speaker_ver1a_model.h5")
i = temp_model(mfccs)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28_input (InputLayer) [(None, 126, 40, 1)]      0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 124, 38, 128)      1280      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 62, 19, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 60, 17, 64)        73792     
_________________________________________________________________
flatten_21 (Flatten)         (None, 65280)             0         
Total params: 75,072
Trainable params: 75,072
Non-trainable params: 0
_________________________________________________________________


NameError: ignored

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cosine
def c_distance(arr1, arr2):
  print(cosine(np.reshape(arr1, (1,-1)), np.reshape(arr2, (1,-1))))


c_distance(i[0], i[1])
c_distance(i[0], i[2])
c_distance(i[0], i[3])
c_distance(i[1], i[2])
c_distance(i[1], i[3])
c_distance(i[2], i[3])

[[0.9189786]]
[[0.8490906]]
[[0.89874595]]
[[0.891236]]
[[0.89851266]]
[[0.9100953]]


In [ ]:
def e_distance(arr1, arr2):
  print( np.sum(np.square(arr1-arr2))/(arr1.shape[0] ))


e_distance(i[0], i[1])
e_distance(i[0], i[2])
e_distance(i[0], i[3])
e_distance(i[1], i[2])
e_distance(i[1], i[3])
e_distance(i[2], i[3])

6.047975083563243
10.746620165269524
7.393851469651953
7.3969627082449065
6.884150092184847
5.137544019922538
